In [ ]:
import BenchmarkTools

In [ ]:
import Random

In [ ]:
import Profile

# Main functions
It would be nice to add a "stack" that keeps track of which changes were made when, going from ancestral to current sequence.

In [ ]:
function MutateSNP!(Sequence::Array{Char}, MutationRate::Real)
	# pick the site to mutate
	sites_to_mutate = map(i -> i < MutationRate, rand(length(Sequence)))
	@inbounds Sequence[sites_to_mutate] .= Random.rand(['A', 'C', 'G', 'T'])
	return Sequence
end

In [ ]:
function MutateInDel!(Sequence::Vector{Char}, InDelRate::Real)
	# loop over every position in the sequence starting with 1
	i = 1
	while i <= length(Sequence)
		# get a random number
		rand_val = Random.rand()
		# if the number is less than half the indel rate, add a char
		if rand_val < InDelRate/2
			insert!(Sequence, i, Random.rand(['A', 'C', 'G', 'T']))
		# if the number is less than the indel rate but greater than half, delete the char
		elseif rand_val < InDelRate
			deleteat!(Sequence, i)
		end
		# increment the counter
		i += 1
	end
end

In [ ]:
function Mutate!(Sequence::Array{Char}, MutationRate::Real, InDelRate::Real, Generation)
	for i in range(1, Generation)
		MutateInDel!(MutateSNP!(Sequence, MutationRate), InDelRate)
	end
	return(Sequence)
end

In [1]:
function GenerateSequences(Seq0::Vector{Char}, NumSequences::Integer, NumGenerations::Integer, MutationRate::Real, InDelRate::Real)
	# instantiate a vector with copies of Seq0
	seqs = [copy(Seq0) for i in range(1, NumSequences)]

	# broadcast the mutate function over the items in the array
	seqs .= Mutate!.(seqs, MutationRate, InDelRate, NumGenerations)
	# return an array of character vectors
	return seqs
end

GenerateSequences (generic function with 1 method)

# provide an input string
And get the desired number of mutated strings

In [ ]:
begin
	Random.seed!(12)
	out = GenerateSequences(collect(Random.randstring("ACGT", 200)), 10, 1000, 0.0001, 0.00001)
	out = map(i -> join(i), out)
	out_fmt = join(map(i -> ">seq"*string(i)*"\n"*out[i]*"\n", 1:10))
	print(out_fmt)
end

In [ ]:
@BenchmarkTools.btime GenerateSequences(collect(Random.randstring("ACGT", 200)), 2, 10000, 0.0001, 0.00001)

In [ ]:
# @Profile.profile GenerateSequences(collect(Random.randstring("ACGT", 200)), 2, 10000, 0.0001, 0.00001)

@Profile.profile MutateSNP!(collect(Random.randstring("ACGT", 1000000)), 0.5)

In [ ]:
Profile.print()

# try making a tree-like structure
The next task would be to separate branches in some defined way, and call the above functions to generate sequences that imitate a phylogenetic tree.